<a href="https://colab.research.google.com/github/kiettranmrac/ghack24/blob/main/ML_GHack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [151]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

data = pd.read_csv('mushrooms.csv')
# data


In [152]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn import preprocessing as pre

enc = OrdinalEncoder()
data = enc.fit_transform(data).astype(np.int64)
# df_test = pd.DataFrame(data)
# display(df_test.head(10))
# display(df_test.describe())
arr = data.max(axis=0)
# display(arr)
for i in range(len(data)):
  for j in range(1, len(data[i])):
    if (arr[j] != 0):
      data[i][j] = data[i][j] / arr[j]


# Split data

data = np.array(data)
m, n = data.shape
np.random.shuffle(data) # shuffle before splitting into dev and training sets

data_dev = data[0:6500].T # chose 6500 as ~80% of data for dev
Y_dev = data_dev[0]
X_dev = data_dev[1:n]

data_train = data[6500:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
_,m_train = X_train.shape



In [153]:
display(Y_train.shape)
display(X_train.shape)

(1624,)

(22, 1624)

In [154]:
# Methods for neurel network
def init_params():
    W1 = np.random.rand(2, 22) - 0.5
    b1 = np.random.rand(2, 1) - 0.5
    W2 = np.random.rand(2, 2) - 0.5
    b2 = np.random.rand(2, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 50 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [155]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 500)

Iteration:  0
[1 0 1 ... 1 1 1] [0 0 1 ... 0 1 0]
0.4544334975369458
Iteration:  50
[0 0 1 ... 1 1 1] [0 0 1 ... 0 1 0]
0.4963054187192118
Iteration:  100
[0 0 1 ... 0 1 0] [0 0 1 ... 0 1 0]
0.5763546798029556
Iteration:  150
[0 0 1 ... 0 1 0] [0 0 1 ... 0 1 0]
0.6681034482758621
Iteration:  200
[0 0 1 ... 0 1 0] [0 0 1 ... 0 1 0]
0.728448275862069
Iteration:  250
[0 0 1 ... 0 1 1] [0 0 1 ... 0 1 0]
0.6908866995073891
Iteration:  300
[0 0 1 ... 0 1 1] [0 0 1 ... 0 1 0]
0.7333743842364532
Iteration:  350
[0 0 1 ... 0 1 0] [0 0 1 ... 0 1 0]
0.8325123152709359
Iteration:  400
[0 0 1 ... 0 1 0] [0 0 1 ... 0 1 0]
0.8448275862068966
Iteration:  450
[0 0 1 ... 0 1 0] [0 0 1 ... 0 1 0]
0.8947044334975369


In [148]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(index, W1, b1, W2, b2):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2)
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)

In [149]:
test_prediction(0, W1, b1, W2, b2)
test_prediction(1, W1, b1, W2, b2)
test_prediction(2, W1, b1, W2, b2)
test_prediction(3, W1, b1, W2, b2)

Prediction:  [1]
Label:  0
Prediction:  [1]
Label:  0
Prediction:  [1]
Label:  1
Prediction:  [1]
Label:  0


In [150]:
dev_predictions = make_predictions(X_dev, W1, b1, W2, b2)
get_accuracy(dev_predictions, Y_dev)

[1 1 1 ... 1 1 1] [1 1 0 ... 1 1 0]


0.4829230769230769